In [82]:
import pandas as pd
import numpy as np
test_data=pd.read_csv("test.csv")
#make pseudo labels:

confident_ids=[]
predicted= pd.read_csv('Tony_final_ensemble_ver3.csv') #best log loss currently 3.5


print(predicted.columns[1:])

confidence_threshold = 0.95
# confident_indices = np.max(predicted[predicted.columns[1:]], axis=1) > confidence_threshold
confident_indices = predicted[predicted.columns[1:]].max(axis=1) > confidence_threshold

confident_ids = predicted.loc[confident_indices, 'id'].tolist()

# Create pseudo labels by finding the column (class) with the highest probability for each confident ID
pseudo_labels = predicted.loc[confident_indices, predicted.columns[1:]].idxmax(axis=1)

# Combine the IDs and their corresponding pseudo labels into a new DataFrame
pseudo_label_df = pd.DataFrame({
    'id': predicted.loc[confident_indices, 'id'],
    'pseudo_label': pseudo_labels
})

# Display or use the pseudo-label DataFrame
print(pseudo_label_df)

confident_ids
print(len(confident_ids))

pseudo_label_df

Index(['Status_C', 'Status_CL', 'Status_D'], dtype='object')
         id pseudo_label
0     15000     Status_D
5     15005     Status_C
6     15006     Status_C
7     15007     Status_C
11    15011     Status_C
...     ...          ...
9991  24991     Status_C
9992  24992     Status_C
9996  24996     Status_C
9997  24997     Status_D
9999  24999     Status_D

[3564 rows x 2 columns]
3564


,id,pseudo_label
0,15000,Status_D
5,15005,Status_C
6,15006,Status_C
7,15007,Status_C
11,15011,Status_C
...,...,...
9991,24991,Status_C
9992,24992,Status_C
9996,24996,Status_C
9997,24997,Status_D


In [83]:
pseudo_label_df

,id,pseudo_label
0,15000,Status_D
5,15005,Status_C
6,15006,Status_C
7,15007,Status_C
11,15011,Status_C
...,...,...
9991,24991,Status_C
9992,24992,Status_C
9996,24996,Status_C
9997,24997,Status_D


In [84]:
pseudo_test_data = test_data[test_data['id'].isin(confident_ids)]
pseudo_test_data


,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,15000,130.0,D-penicillamine,16944.0,F,Y,Y,Y,N,17.4,NaN,3.00,182.0,559.0,119.35,NaN,401.0,11.0,4.0
5,15005,2576.0,Placebo,16122.0,F,N,Y,N,N,0.5,256.0,3.87,14.0,423.0,49.60,84.0,336.0,10.4,3.0
6,15006,2863.0,D-penicillamine,20736.0,F,N,N,N,N,0.6,251.0,4.38,12.0,674.0,57.00,85.0,344.0,9.9,3.0
7,15007,2178.0,D-penicillamine,16858.0,F,N,N,N,N,0.8,172.0,3.74,35.0,1134.0,71.30,96.0,311.0,10.6,3.0
11,15011,1250.0,NaN,13149.0,F,NaN,NaN,NaN,N,1.6,NaN,3.50,NaN,NaN,NaN,NaN,150.0,10.6,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,24991,2149.0,D-penicillamine,22336.0,F,N,N,N,N,0.6,NaN,3.89,20.0,678.0,58.00,NaN,128.0,10.0,2.0
9992,24992,1433.0,Placebo,20510.0,F,N,N,N,N,0.7,298.0,4.01,28.0,733.0,65.10,58.0,256.0,10.7,2.0
9996,24996,2580.0,D-penicillamine,25569.0,F,N,N,N,N,0.4,NaN,4.01,20.0,666.0,54.25,NaN,277.0,10.0,3.0
9997,24997,186.0,Placebo,21483.0,F,N,Y,Y,S,6.6,1000.0,3.50,188.0,944.0,130.20,133.0,265.0,11.0,4.0


In [85]:

pseudo_label_df
# from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
# le = LabelEncoder()
# pseudo_label = le.fit_transform(pseudo_labels)  # 'C' -> 0, 'D' -> 1, 'CL' -> 2

# pseudo_label

print(pseudo_label_df['pseudo_label'].unique())

# Your original status array
status_array = pseudo_label_df['pseudo_label']

# Create a mapping dictionary
status_mapping = {
    'Status_C':'C',
    'Status_D':'D',
    'Status_CL':'CL'
}

# Use a list comprehension to transform the array based on the mapping
transformed_status = np.array([status_mapping.get(item, item) for item in status_array])

# Display the transformed array
print(transformed_status)

pseudo_test_label=transformed_status


['Status_D' 'Status_C']
['D' 'C' 'C' ... 'C' 'D' 'D']


In [86]:
pseudo_test_data['Status']=pseudo_test_label
pseudo_test_data

total_pseudo_data = pd.concat([train_data, pseudo_test_data], ignore_index=True)
total_pseudo_data

np.save('total_pseudo_data_95per',total_pseudo_data)
np.load('total_pseudo_data_95per.npy',allow_pickle=True)

C:\Users\tonys\AppData\Local\Temp\ipykernel_9332\892407187.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pseudo_test_data['Status']=pseudo_test_label


array([[0, 3157.0, nan, ..., 10.9, 4.0, 'C'],
       [1, 1568.0, 'Placebo', ..., 12.7, 4.0, 'D'],
       [2, 1367.0, nan, ..., 11.3, 4.0, 'D'],
       ...,
       [24996, 2580.0, 'D-penicillamine', ..., 10.0, 3.0, 'C'],
       [24997, 186.0, 'Placebo', ..., 11.0, 4.0, 'D'],
       [24999, 778.0, nan, ..., 11.5, 4.0, 'D']], dtype=object)

In [64]:

le = LabelEncoder()
pseudo_train_label = le.fit_transform(transformed_status)  # 'C' -> 0, 'D' -> 1, 'CL' -> 2
print(pseudo_train_label)

[1 0 0 ... 0 1 1]


In [67]:
# Load your dataset
train_data = pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")
train_label=train_data['Status']

In [68]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

# Define categorical and continuous columns
cat_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema','Stage']
cont_cols = ['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides','Platelets','Prothrombin']

print(len(cat_cols) + len(cont_cols) == 18, f': if True then Good, total {len(cat_cols) + len(cont_cols)} features')

# Preprocessing function for XGBoost
def random_preprocess_for_xgboost(df):
    # Continuous variables handling
    X_cont = df[cont_cols].copy()
    for feat in cont_cols:
        non_nan_values = X_cont[feat].dropna().values
        X_cont[feat] = X_cont[feat].apply(lambda x: np.random.choice(non_nan_values) if pd.isna(x) else x)
    
    # Categorical variables handling
    X_cat = df[cat_cols].copy()
    for col in cat_cols:
        le = LabelEncoder()
        # Fit the label encoder
        X_cat[col] = le.fit_transform(X_cat[col].astype(str))
        # Add a new label for missing values (next integer value)
        max_label = X_cat[col].max() + 1
        X_cat[col] = X_cat[col].fillna(max_label)
    
    # Combine continuous and categorical variables
    X_xgb = pd.concat([X_cont, X_cat], axis=1)
    
    return X_xgb

# Apply preprocessing to the train_data
X_random_xgb = random_preprocess_for_xgboost(train_data)


True : if True then Good, total 18 features


In [69]:
train_data

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,3157.0,NaN,24472.0,F,NaN,NaN,NaN,N,1.9,NaN,3.81,NaN,NaN,NaN,NaN,141.0,10.9,4.0,C
1,1,1568.0,Placebo,19698.0,F,Y,Y,N,Y,2.5,178.0,2.56,209.0,815.0,159.65,78.0,149.0,12.7,4.0,D
2,2,1367.0,NaN,20819.0,F,NaN,NaN,NaN,N,2.0,NaN,3.05,NaN,NaN,NaN,NaN,80.0,11.3,4.0,D
3,3,1092.0,NaN,14610.0,F,NaN,NaN,NaN,N,2.9,NaN,3.73,NaN,NaN,NaN,NaN,337.0,10.2,4.0,C
4,4,1980.0,NaN,18628.0,F,NaN,NaN,NaN,N,0.5,NaN,3.12,NaN,NaN,NaN,NaN,190.0,11.2,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,1328.0,NaN,23376.0,F,NaN,NaN,NaN,N,0.7,NaN,3.14,NaN,NaN,NaN,NaN,425.0,10.9,3.0,D
14996,14996,904.0,D-penicillamine,22336.0,F,N,N,Y,N,0.6,396.0,3.53,102.0,1257.0,137.95,118.0,216.0,10.6,1.0,D
14997,14997,989.0,NaN,23376.0,F,NaN,NaN,NaN,N,0.6,NaN,3.40,NaN,NaN,NaN,NaN,388.0,11.0,3.0,C
14998,14998,790.0,Placebo,19994.0,F,N,Y,N,N,3.2,NaN,3.41,86.0,1790.0,134.85,NaN,149.0,11.0,4.0,D
